# Train XGBoost with cat_in_the_dat dataset

In [6]:
import os
from tempfile import TemporaryDirectory
from time import time

import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

import xgboost as xgb


def load_cat_in_the_dat() -> tuple[pd.DataFrame, pd.Series]:
    """Assuming you have already downloaded the data into `input` directory."""

    df_train = pd.read_csv("./cat-in-the-dat/train.csv")

    print(
        "train data set has got {} rows and {} columns".format(
            df_train.shape[0], df_train.shape[1]
        )
    )
    X = df_train.drop(["target"], axis=1)
    y = df_train["target"]

    for i in range(0, 5):
        X["bin_" + str(i)] = X["bin_" + str(i)].astype("category")

    for i in range(0, 5):
        X["nom_" + str(i)] = X["nom_" + str(i)].astype("category")

    for i in range(5, 10):
        X["nom_" + str(i)] = X["nom_" + str(i)].apply(int, base=16)

    for i in range(0, 6):
        X["ord_" + str(i)] = X["ord_" + str(i)].astype("category")

    print(
        "train data set has got {} rows and {} columns".format(X.shape[0], X.shape[1])
    )
    return X, y


params = {
    "tree_method": "hist",
    "device": "cuda",
    "n_estimators": 32,
    "colsample_bylevel": 0.7,
}


def categorical_model(X: pd.DataFrame, y: pd.Series, output_dir: str) -> None:
    """Train using builtin categorical data support from XGBoost"""
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1994, test_size=0.2
    )
    # Be aware that the encoding for X_train and X_test are the same here. In practice,
    # we should try to use an encoder like (sklearn OrdinalEncoder) to obtain the
    # categorical values.

    # Specify `enable_categorical` to True.
    clf = xgb.XGBClassifier(
        **params,
        eval_metric="auc",
        enable_categorical=True,
        max_cat_to_onehot=1,  # We use optimal partitioning exclusively
    )
    clf.fit(X_train, y_train, eval_set=[(X_test, y_test), (X_train, y_train)])
    clf.save_model(os.path.join(output_dir, "categorical.json"))

    y_score = clf.predict_proba(X_test)[:, 1]  # proba of positive samples
    auc = roc_auc_score(y_test, y_score)
    print("AUC of using builtin categorical data support:", auc)


def onehot_encoding_model(X: pd.DataFrame, y: pd.Series, output_dir: str) -> None:
    """Train using one-hot encoded data."""
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=42, test_size=0.2
    )
    # Specify `enable_categorical` to False as we are using encoded data.
    clf = xgb.XGBClassifier(**params, eval_metric="auc", enable_categorical=False)
    clf.fit(
        X_train,
        y_train,
        eval_set=[(X_test, y_test), (X_train, y_train)],
    )
    clf.save_model(os.path.join(output_dir, "one-hot.json"))

    y_score = clf.predict_proba(X_test)[:, 1]  # proba of positive samples
    auc = roc_auc_score(y_test, y_score)
    print("AUC of using onehot encoding:", auc)

In [7]:
X, y = load_cat_in_the_dat()

with TemporaryDirectory() as tmpdir:
    start = time()
    categorical_model(X, y, tmpdir)
    end = time()
    print("Duration:categorical", end - start)

    X = pd.get_dummies(X)
    start = time()
    onehot_encoding_model(X, y, tmpdir)
    end = time()
    print("Duration:onehot", end - start)

train data set has got 300000 rows and 25 columns
train data set has got 300000 rows and 24 columns
[23:27:37] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1700181009589/work/src/learner.cc:767: 
Parameters: { "device" } are not used.

[0]	validation_0-auc:0.69738	validation_1-auc:0.70022
[1]	validation_0-auc:0.71226	validation_1-auc:0.71859
[2]	validation_0-auc:0.71756	validation_1-auc:0.72727
[3]	validation_0-auc:0.72711	validation_1-auc:0.73959
[4]	validation_0-auc:0.73034	validation_1-auc:0.74650
[5]	validation_0-auc:0.73400	validation_1-auc:0.75297
[6]	validation_0-auc:0.73659	validation_1-auc:0.76000
[7]	validation_0-auc:0.73906	validation_1-auc:0.76634
[8]	validation_0-auc:0.74167	validation_1-auc:0.77111
[9]	validation_0-auc:0.74362	validation_1-auc:0.77643
[10]	validation_0-auc:0.74472	validation_1-auc:0.77894
[11]	validation_0-auc:0.74628	validation_1-auc:0.78330
[12]	validation_0-auc:0.74772	validation_1-auc:0.78555
[13]	validation_0-auc:0.74814	validation_1-auc